In [42]:
# !pip install pandas sqlite3

In [43]:
import sqlite3
import pandas as pd

In [44]:
csv_file = 'MoviesDataset.csv' 
df = pd.read_csv(csv_file)

In [45]:
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS Movies (
    title TEXT,
    genre TEXT,
    rating REAL,
    runtime INTEGER,
    gross_income TEXT,
    year INTEGER,
    starcast TEXT
)
''')

In [46]:
df.to_sql('Movies', conn, if_exists='replace', index=False)


5562

In [47]:
cursor.execute("DELETE FROM Movies WHERE rating < 5.0")
conn.commit()

rem_movies = cursor.execute("SELECT COUNT(*) FROM Movies").fetchone()[0]
print(f"The remaining movies are: {rem_movies}")

The remaining movies are: 5386


In [48]:
unique_genres_combined = cursor.execute('''
    SELECT COUNT(DISTINCT genre)
    FROM (
        SELECT main_genre AS genre FROM Movies
        UNION
        SELECT side_genre AS genre FROM Movies
    )
''').fetchone()[0]
print(f"Unique Combined Genres: {unique_genres_combined}")

Unique Combined Genres: 152


In [49]:
avg_rt = cursor.execute('''
    SELECT AVG("Runtime(Mins)") 
    FROM Movies 
    WHERE rating BETWEEN 8 AND 9
''').fetchone()[0]
print(f"Average Runtime for Movies with Rating 8-9: {avg_rt}")

Average Runtime for Movies with Rating 8-9: 130.92616033755274


In [50]:
top_5_gross_income = cursor.execute('''
    SELECT "Total_Gross" 
    FROM Movies 
    WHERE year = 2000 AND "Total_Gross" IS NOT NULL 
    ORDER BY rating DESC 
    LIMIT 5
''').fetchall()


gross_income_formatted = ', '.join(row[0] for row in top_5_gross_income)
print(f"Top 5 Gross income movies from 2000): {gross_income_formatted}")

Top 5 Gross income movies from 2000): $187.71M, $25.54M, $3.64M, $30.33M, Gross Unkown


In [51]:
columns = cursor.execute("PRAGMA table_info(Movies)").fetchall()
print(columns)

[(0, 'Movie_Title', 'TEXT', 0, None, 0), (1, 'Year', 'INTEGER', 0, None, 0), (2, 'Director', 'TEXT', 0, None, 0), (3, 'Actors', 'TEXT', 0, None, 0), (4, 'Rating', 'REAL', 0, None, 0), (5, 'Runtime(Mins)', 'INTEGER', 0, None, 0), (6, 'Censor', 'TEXT', 0, None, 0), (7, 'Total_Gross', 'TEXT', 0, None, 0), (8, 'main_genre', 'TEXT', 0, None, 0), (9, 'side_genre', 'TEXT', 0, None, 0)]


In [52]:
chris_evans_movies = cursor.execute('''
    SELECT Movie_Title 
    FROM Movies 
    WHERE "Starcast" LIKE '%Chris Evans%'
''').fetchall()

robert_downey_jr_movies = cursor.execute('''
    SELECT Movie_Title 
    FROM Movies 
    WHERE "Starcast" LIKE '%Robert Downey Jr.%'
''').fetchall()

print("Movies with Chris Evans:", [row[0] for row in chris_evans_movies])
print("Movies with Robert Downey Jr.:", [row[0] for row in robert_downey_jr_movies])

Movies with Chris Evans: []
Movies with Robert Downey Jr.: []


In [53]:
movies_with_chris_and_robert = cursor.execute('''
    SELECT Movie_Title 
    FROM Movies 
    WHERE "Starcast" LIKE '%Chris Evans%' 
    AND "Starcast" LIKE '%Robert Downey Jr.%'
''').fetchall()

movies_titles = [row[0] for row in movies_with_chris_and_robert]
print(f"Movies with Chris Evans and Robert Downey Jr.: {', '.join(movies_titles) if movies_titles else 'None found'}")

Movies with Chris Evans and Robert Downey Jr.: None found


In [54]:
columns = cursor.execute("PRAGMA table_info(Movies)").fetchall()
print(columns)

sample_data = cursor.execute("SELECT * FROM Movies LIMIT 10").fetchall()
print(sample_data)

[(0, 'Movie_Title', 'TEXT', 0, None, 0), (1, 'Year', 'INTEGER', 0, None, 0), (2, 'Director', 'TEXT', 0, None, 0), (3, 'Actors', 'TEXT', 0, None, 0), (4, 'Rating', 'REAL', 0, None, 0), (5, 'Runtime(Mins)', 'INTEGER', 0, None, 0), (6, 'Censor', 'TEXT', 0, None, 0), (7, 'Total_Gross', 'TEXT', 0, None, 0), (8, 'main_genre', 'TEXT', 0, None, 0), (9, 'side_genre', 'TEXT', 0, None, 0)]
[('Kantara', 2022, 'Rishab Shetty', 'Rishab Shetty, Sapthami Gowda, Kishore Kumar G., Achyuth Kumar', 9.3, 148, 'UA', 'Gross Unkown', 'Action', ' Adventure,  Drama'), ('The Dark Knight', 2008, 'Christopher Nolan', 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 9.0, 152, 'UA', '$534.86M', 'Action', ' Crime,  Drama'), ('The Lord of the Rings: The Return of the King', 2003, 'Peter Jackson', 'Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom', 9.0, 201, 'U', '$377.85M', 'Action', ' Adventure,  Drama'), ('Inception', 2010, 'Christopher Nolan', 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot P